# Walk Convert

Iterate through gps-walks folder, limiting to one point every 5 seconds, converting to GPX + CSV format.

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys

import numpy as np

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader
from file_writer import getFileWriter

from track import Track

## Main Function

In [2]:
TIMESTAMP = 'ts'

def getTsDiff(track):
    '''Determine typical timestamp difference for the track'''

    tsDiff = None
    
    if TIMESTAMP in track.data:
        tsDiffs = np.round(track.data[TIMESTAMP][1:] - track.data[TIMESTAMP][:-1], 2)
        tsDiff = np.median(tsDiffs)

    return tsDiff


def sampleTracks(fileReader):
    '''Create samples for each of the tracks, limiting to once every 5 seconds'''

    samples = []

    for track in fileReader.tracks:
        # Determine the step size equivalent to 5 seconds
        tsDiff = getTsDiff(track)
        if tsDiff:
            step = round(5 / tsDiff)
        else:
            step = 1

        # Create a new track for the sample
        sample = Track()
        for field in ['ts', 'lat', 'lon', 'ele', 'sog', 'cog']:
            if field in track.fields:
                sample.fields[field] = track.fields[field]

            if field in track.data:
                sample.data[field] = track.data[field][::step]
                sample.numPoints = len(sample.data[field])

        samples.append(sample)

    return samples


def batchConvert():
    '''Iterate through session archive testing each GPS file'''

    dataDir = os.path.normpath(os.path.join(projdir, '..', 'gps-walks', 'data'))
    rawDir = os.path.join(dataDir, 'raw')

    errors = {}

    for root, subDirs, files in os.walk(rawDir):
        # Ignore backup folders
        if os.path.split(root)[1] != 'backup':
            for file in files:
                ext = os.path.splitext(file)[1].lower()
                
                if ext and ext in ['.sbp', '.sbn', '.gpx', '.fit']:
                    try:
                        # Initialise file reader but postpone loading tha data
                        filePath = os.path.join(root, file)
                        fileReader = getFileReader(filePath)
    
                        for outFormat in ['gpx', 'kml', 'csv']:
                            # Create output directory if it does not already exist
                            outDir = root.replace(rawDir, os.path.join(dataDir, outFormat))
                            if not os.path.exists(outDir):
                                os.makedirs(outDir)
    
                            # Output filename is the same as the original but with a different extension
                            outFile = '{}.{}'.format(os.path.splitext(file)[0], outFormat)
    
                            # Only generate the output file if it does not already exist
                            outPath = os.path.join(outDir, outFile)
                            if not os.path.exists(outPath):
                                # The original track may not have been loaded yet
                                if not fileReader.tracks:
                                    print('Loading ', file)
                                    fileReader.load()
    
                                    samples = sampleTracks(fileReader)
    
                                print('Saving ', outFile)
                                fileWriter = getFileWriter(outPath, tracks=samples)
                                fileWriter.save()
    
                        #print('.', end='')
    
                    except Exception:
                        errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
    
                        print('E', end='')

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for filename, error in errors.items():
            print(filename)
            print(error)

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    batchConvert()
    
    print(os.linesep + 'All done!')


All done!
